In [1]:
#encoding=utf-8
import pandas as pd
import numpy as np
import re
import string

import jieba
jieba.load_userdict("../../code/WordCut/userdict.txt")

import gc

import pickle

# get data
model_list = ['IDClassifier', 'IfKnowDebtor', 'ConfirmLoan', 'WillingToPay', 'CutDebt', 'Installment']
# path = '/home/kai/data/jiangning/Chatbot_1/Chatbot1.0/data/'
path = '../../data/others/'
data = pd.read_csv(path + 'irrelevant_response_training_set.csv', encoding='utf8')
strategy_mat = pd.read_csv(path + 'irrelevant_response_strategy_matrix.csv', encoding='utf8')

print(data.shape)
print(strategy_mat.shape)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.912 seconds.
Prefix dict has been built succesfully.


(2054, 2)
(13, 8)


# data cleaning

In [58]:
data = data.rename(index=str, columns={'文本': 'text', '类别': 'label'})
strategy_mat = strategy_mat.rename(index=str, columns={'类别': 'label', '对策': 'strategy', 'Idclassfier':'IDClassifier', 'ifknowdebtor': 'IfKnowDebtor', 'willingtopay':'WillingToPay', 'cutdebt': 'CutDebt', 'installment': 'Installment'})

print(data['label'].unique())
print(data.label.value_counts())

def clean_label(label):
    coder = {'讨价还价':0, '说出目的':1, '确认数额':2, '请求重复':3, '请求等下打来':4, '其它通讯方式':5, '模糊确认':6, '回问身份':7, '还款方式':8, '故意岔开话题':9, '不愿配合':10}
    pickle.dump(coder, open('../../data/others/label_token.pkl','wb'))
    return coder[label]

def cut_words(text):
    seg_list = jieba.cut(text, cut_all=False)
    return " ".join(seg_list)

def clean(text):
    text = re.sub(f'([{string.punctuation}“”¨«»®´·º ½¾¿¡§£₤‘’，])',' ', text)
    text = text.split(' ')
    text = ' '.join(text)
    return text



    
# cut words
data['text']=data['text'].apply(cut_words)
print('finish cutting words')

# cleaning and save
data['text'] = data['text'].apply(clean)
data['label'] = data['label'].apply(clean_label)

# shuffle data
data = data.sample(frac=1).reset_index(drop=True)
print('finish shuffling')
data.to_csv(path.format(each_model) + 'cleaned_mock_up_data.csv', index = False, encoding = 'utf8')

strategy_mat['label'] = strategy_mat['label'].apply(clean_label)



['讨价还价' '说出目的' '确认数额' '请求重复' '请求等下打来' '其它通讯方式' '模糊确认' '回问身份' '还款方式'
 '故意岔开话题' '不愿配合']
请求等下打来    612
讨价还价      489
请求重复      390
故意岔开话题    231
不愿配合      132
回问身份       69
说出目的       54
模糊确认       40
确认数额       24
还款方式       10
其它通讯方式      3
Name: label, dtype: int64
finish cutting words
finish shuffling


In [59]:
data.head(10)

,text,label
0,天气 怎样 啊,9
1,过 四分钟 打 来,4
2,有事吗,1
3,我 现在 睡觉 没空 。,4
4,说 啥 呢,3
5,啥 ？ 免 啥 ？,3
6,雨太大 了,4
7,下班 以后 再 联系,4
8,钱 凑 好 了 联系 你,0
9,再 想想,0


In [2]:
strategy_mat

,类别,Idclassfier,ifknowdebtor,ConfirmLoan,willingtopay,cutdebt,installment,对策
0,讨价还价,1.0,1.0,0.0,1.0,0.0,0.0,cutdebt
1,讨价还价,1.0,1.0,1.0,1.0,0.0,0.0,施压
2,说出目的,0.0,0.0,1.0,1.0,1.0,1.0,ConfirmLoan
3,确认数额,1.0,1.0,0.0,1.0,1.0,1.0,willingtopay
4,请求重复,0.0,0.0,0.0,0.0,0.0,0.0,重复
5,请求等下打来,0.0,0.0,0.0,0.0,0.0,0.0,另选时间打来
6,其它通讯方式,1.0,1.0,1.0,0.0,1.0,1.0,重复
7,模糊确认,1.0,1.0,0.0,0.0,0.0,0.0,施压
8,回问身份,1.0,1.0,0.0,0.0,0.0,0.0,施压
9,回问身份,0.0,0.0,1.0,1.0,1.0,1.0,重复


# modeling

In [74]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import MultinomialNB

from others_py import *

func_list = [CutDebt_other,IDClassifier_other,IfKnowDebtor_other,Installment_other,ConfirmLoan_other,WillingToPay_other]

In [77]:
ind = 0
for each_model in model_list:
    print(each_model)
    possible_label = list(set(strategy_mat[strategy_mat[each_model]==0]['label'].values))
    print(possible_label)
    train_data = data[data['label'].apply(lambda x: x in possible_label)]
    
    # get tfidf
    phrase_vectorizer = TfidfVectorizer(ngram_range=(1,3),
                                    strip_accents='unicode', 
                                    max_features=100000, 
                                    analyzer='word',
                                    sublinear_tf=True,
                                    token_pattern=r'\w{1,}')

    print('fitting phrase')
    phrase_vectorizer.fit(train_data.text)

    print('transform phrase')
    phrase = phrase_vectorizer.transform(train_data.text)

    
    # linear svc
    l_svc = LinearSVC()
    lsvc = CalibratedClassifierCV(l_svc) 
    lsvc.fit(phrase, train_data.label)
    
    
    # logistic
    log_r = LogisticRegression()
    log_r.fit(phrase, train_data.label)
    
    
    # Naive Bayes
    naive_b = MultinomialNB()
    naive_b.fit(phrase, train_data.label)
    
    print('finish training')
    
    
    # save model
    save_path = '../../savedModel/others/{}/'
#     # save tfidf
#     pickle.dump(phrase_vectorizer, open(save_path.format(each_model) + "tfidf.pickle", "wb"))
#     # save linear svc
#     pickle.dump(lsvc, open(save_path.format(each_model) + "LinearSVC.pickle", "wb"))
#     # save logistic
#     pickle.dump(log_r, open(save_path.format(each_model) + "Logistic.pickle", "wb"))
#     # save naive bayes
#     pickle.dump(naive_b, open(save_path.format(each_model) + "nb.pickle", "wb"))
    
    
    func = func_list[ind]
    result = func(svc=lsvc, logistic=log_r, nb=naive_b, tfidf=phrase_vectorizer, jieba_path='../WordCut/userdict.txt')
    pickle.dump(result, open(save_path.format(each_model) + each_model + '_other.pickle', "wb"))
    ind = ind + 1
    
   

IDClassifier
[1, 3, 4, 7, 9, 10]
fitting phrase
transform phrase
finish training
IfKnowDebtor
[1, 3, 4, 7, 9, 10]
fitting phrase
transform phrase
finish training
ConfirmLoan
[0, 2, 3, 4, 6, 7, 8, 9]
fitting phrase
transform phrase
finish training
WillingToPay
[3, 4, 5, 6, 7, 8, 9]
fitting phrase
transform phrase
finish training
CutDebt
[0, 3, 4, 6, 7, 8, 9]
fitting phrase
transform phrase
finish training
Installment
[0, 3, 4, 6, 7, 8, 9]
fitting phrase
transform phrase
finish training


In [78]:
# coder = {'讨价还价':0, '说出目的':1, '确认数额':2, '请求重复':3, '请求等下打来':4, '其它通讯方式':5, '模糊确认':6, '回问身份':7, '还款方式':8, '故意岔开话题':9, '不愿配合':10}

idc = pickle.load(open("../../savedModel/others/IDClassifier/IDClassifier_other.pickle", 'rb'))
cutd = pickle.load(open("../../savedModel/others/CutDebt/CutDebt_other.pickle", 'rb'))
ifk = pickle.load(open("../../savedModel/others/IfKnowDebtor/IfKnowDebtor_other.pickle", 'rb'))
will = pickle.load(open("../../savedModel/others/WillingToPay/WillingToPay_other.pickle", 'rb'))
inst = pickle.load(open("../../savedModel/others/Installment/Installment_other.pickle", 'rb'))
conf = pickle.load(open("../../savedModel/others/ConfirmLoan/ConfirmLoan_other.pickle", 'rb'))

In [81]:
idc.classify('老子不还了')

{'label': 2,
 'pred_prob': array([[0.009732  , 0.00695486, 0.12269119, 0.00142007, 0.34699846,
         0.51220342],
        [0.03303455, 0.13671193, 0.30620658, 0.02035326, 0.2740939 ,
         0.22959977],
        [0.00814982, 0.18384136, 0.59789614, 0.00752273, 0.12899894,
         0.07359101]]),
 'av_pred': array([0.01697212, 0.10916938, 0.34226464, 0.00976535, 0.25003043,
        0.27179807])}

In [54]:
# a = pickle.load(open('../../data/others/label_token.pkl','rb'))